In [2]:
from google.colab import files

files.upload()

Saving input_roots.tar.gz to input_roots.tar.gz


In [24]:
ls

input_1.pkl  input_4.pkl         roots_1.pkl  roots_4.pkl
input_2.pkl  input_5.pkl         roots_2.pkl  roots_5.pkl
input_3.pkl  input_roots.tar.gz  roots_3.pkl  sample_data/


In [26]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers

from keras.models import Sequential
from keras.layers import Dense


X = pd.read_pickle('input_4.pkl')
Y = pd.read_pickle('roots_4.pkl')


print("X head")
print(X.head())

print("\nY head")
print(Y.head())


X_np = X.to_numpy()
Y_np = Y.to_numpy()

DEGREE = X_np.shape[1] - 1


X_train = X_np[:80000]
Y_train = Y_np[:80000]

X_test = X_np[80000:]
Y_test = Y_np[80000:]

print("\n\nX_train, Y_train")
print(X_train.shape, Y_train.shape)
print("\nX_test, Y_test")
print(X_test.shape, Y_test.shape)


def reshape(array):
    return np.expand_dims(array, -1)
    
print("\n\nInput shape")    
print(reshape(X_test).shape) 
print("\nOutput shape")
print(reshape(Y_test).shape)

X head
         a0        a1        a2        a3        a4        a5
0  0.122622  0.674274  0.334432  0.081705  0.002616  0.882685
1  0.523378  0.105365  0.918005  0.539328  0.556930  0.296065
2  0.587407  0.475264  0.904323  0.044569  0.829932  0.048859
3  0.241942  0.485113  0.566889  0.240382  0.593640  0.335084
4  0.668968  0.080742  0.263383  0.612457  0.023483  0.654194

Y head
   Re(alpha1)  Im(alpha1)  Re(alpha2)  ...  Im(alpha4)  Re(alpha5)  Im(alpha5)
0   -0.200382         0.0   -0.581798  ...   -0.775552    0.680507    0.775552
1   -1.878397         0.0   -0.240955  ...   -0.799562    0.239602    0.799562
2  -16.994748         0.0   -0.427214  ...   -1.012390    0.431500    1.012390
3   -1.723917         0.0   -0.460207  ...   -0.917885    0.436357    0.917885
4   -0.897553         0.0   -0.180344  ...   -0.756296    0.611173    0.756296

[5 rows x 10 columns]


X_train, Y_train
(80000, 6) (80000, 10)

X_test, Y_test
(20000, 6) (20000, 10)


Input shape
(20000, 6, 1)

Output

In [12]:
from keras.models import Sequential
from keras.layers import LSTM, RepeatVector, Dense, TimeDistributed


hidden_size = 128
model = Sequential()

# ENCODER PART OF SEQ2SEQ
model.add(LSTM(hidden_size, input_shape=(DEGREE+1, 1)))

# DECODER PART OF SEQ2SEQ
model.add(RepeatVector(DEGREE * 2))  # this determines the length of the output sequence
model.add((LSTM(hidden_size, return_sequences=True)))
model.add(TimeDistributed(Dense(1)))

model.compile(loss='mean_absolute_error',
              optimizer='adam',
              metrics=['mae'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               66560     
_________________________________________________________________
repeat_vector (RepeatVector) (None, 10, 128)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 10, 128)           131584    
_________________________________________________________________
time_distributed (TimeDistri (None, 10, 1)             129       
Total params: 198,273
Trainable params: 198,273
Non-trainable params: 0
_________________________________________________________________
None


In [13]:
# 1st dataset
BATCH_SIZE = 128
model.fit(reshape(X_train), reshape(Y_train), batch_size=BATCH_SIZE, epochs=10, verbose=1, validation_data=(reshape(X_test), reshape(Y_test)))

Epoch 1/10
625/625 [==============================] - 63s 67ms/step - loss: 0.8575 - mae: 0.8575 - val_loss: 0.6112 - val_mae: 0.6112
Epoch 2/10
625/625 [==============================] - 39s 63ms/step - loss: 0.6885 - mae: 0.6885 - val_loss: 0.5371 - val_mae: 0.5371
Epoch 3/10
625/625 [==============================] - 39s 63ms/step - loss: 0.4202 - mae: 0.4202 - val_loss: 0.4772 - val_mae: 0.4772
Epoch 4/10
625/625 [==============================] - 39s 62ms/step - loss: 0.4709 - mae: 0.4709 - val_loss: 0.4668 - val_mae: 0.4668
Epoch 5/10
625/625 [==============================] - 39s 62ms/step - loss: 0.4241 - mae: 0.4241 - val_loss: 0.4426 - val_mae: 0.4426
Epoch 6/10
625/625 [==============================] - 39s 63ms/step - loss: 0.4345 - mae: 0.4345 - val_loss: 0.4275 - val_mae: 0.4275
Epoch 7/10
625/625 [==============================] - 40s 63ms/step - loss: 0.4787 - mae: 0.4787 - val_loss: 0.4188 - val_mae: 0.4188
Epoch 8/10
625/625 [==============================] - 40s 64ms

In [15]:
# 2nd dataset
BATCH_SIZE = 128
model.fit(reshape(X_train), reshape(Y_train), batch_size=BATCH_SIZE, epochs=10, verbose=1, validation_data=(reshape(X_test), reshape(Y_test)))

Epoch 1/10
625/625 [==============================] - 39s 63ms/step - loss: 0.4313 - mae: 0.4313 - val_loss: 0.3120 - val_mae: 0.3120
Epoch 2/10
625/625 [==============================] - 40s 63ms/step - loss: 0.4262 - mae: 0.4262 - val_loss: 0.3182 - val_mae: 0.3182
Epoch 3/10
625/625 [==============================] - 40s 63ms/step - loss: 0.4203 - mae: 0.4203 - val_loss: 0.3027 - val_mae: 0.3027
Epoch 4/10
625/625 [==============================] - 40s 64ms/step - loss: 0.4157 - mae: 0.4157 - val_loss: 0.2951 - val_mae: 0.2951
Epoch 5/10
625/625 [==============================] - 39s 63ms/step - loss: 0.4117 - mae: 0.4117 - val_loss: 0.2895 - val_mae: 0.2895
Epoch 6/10
625/625 [==============================] - 39s 63ms/step - loss: 0.4079 - mae: 0.4079 - val_loss: 0.2815 - val_mae: 0.2815
Epoch 7/10
625/625 [==============================] - 40s 64ms/step - loss: 0.4029 - mae: 0.4029 - val_loss: 0.2805 - val_mae: 0.2805
Epoch 8/10
625/625 [==============================] - 40s 63ms

In [20]:
# 3rd dataset
BATCH_SIZE = 128
model.fit(reshape(X_train), reshape(Y_train), batch_size=BATCH_SIZE, epochs=10, verbose=1, validation_data=(reshape(X_test), reshape(Y_test)))

Epoch 1/10
625/625 [==============================] - 39s 63ms/step - loss: 0.2707 - mae: 0.2707 - val_loss: 0.2581 - val_mae: 0.2581
Epoch 2/10
625/625 [==============================] - 39s 62ms/step - loss: 0.2677 - mae: 0.2677 - val_loss: 0.2617 - val_mae: 0.2617
Epoch 3/10
625/625 [==============================] - 39s 63ms/step - loss: 0.2656 - mae: 0.2656 - val_loss: 0.2609 - val_mae: 0.2609
Epoch 4/10
625/625 [==============================] - 39s 63ms/step - loss: 0.2642 - mae: 0.2642 - val_loss: 0.2556 - val_mae: 0.2556
Epoch 5/10
625/625 [==============================] - 40s 64ms/step - loss: 0.2622 - mae: 0.2622 - val_loss: 0.2522 - val_mae: 0.2522
Epoch 6/10
625/625 [==============================] - 40s 64ms/step - loss: 0.2591 - mae: 0.2591 - val_loss: 0.2506 - val_mae: 0.2506
Epoch 7/10
625/625 [==============================] - 40s 64ms/step - loss: 0.2579 - mae: 0.2579 - val_loss: 0.2494 - val_mae: 0.2494
Epoch 8/10
625/625 [==============================] - 40s 64ms

In [22]:
# 4th dataset
BATCH_SIZE = 128
model.fit(reshape(X_train), reshape(Y_train), batch_size=BATCH_SIZE, epochs=10, verbose=1, validation_data=(reshape(X_test), reshape(Y_test)))

Epoch 1/10
625/625 [==============================] - 39s 63ms/step - loss: 0.3914 - mae: 0.3914 - val_loss: 0.4210 - val_mae: 0.4210
Epoch 2/10
625/625 [==============================] - 39s 63ms/step - loss: 0.3907 - mae: 0.3907 - val_loss: 0.4228 - val_mae: 0.4228
Epoch 3/10
625/625 [==============================] - 39s 62ms/step - loss: 0.3880 - mae: 0.3880 - val_loss: 0.4180 - val_mae: 0.4180
Epoch 4/10
625/625 [==============================] - 39s 63ms/step - loss: 0.3870 - mae: 0.3870 - val_loss: 0.4191 - val_mae: 0.4191
Epoch 5/10
625/625 [==============================] - 39s 62ms/step - loss: 0.3850 - mae: 0.3850 - val_loss: 0.4156 - val_mae: 0.4156
Epoch 6/10
625/625 [==============================] - 39s 62ms/step - loss: 0.3835 - mae: 0.3835 - val_loss: 0.4126 - val_mae: 0.4126
Epoch 7/10
625/625 [==============================] - 39s 62ms/step - loss: 0.3812 - mae: 0.3812 - val_loss: 0.4116 - val_mae: 0.4116
Epoch 8/10
625/625 [==============================] - 39s 62ms

In [27]:
y_pred = model.predict(reshape(X_test));

In [33]:
x = 200
y_pred[x] - reshape(Y_test[x])

array([[ 0.0019105 ],
       [ 0.00084141],
       [ 0.00392224],
       [ 0.0041809 ],
       [ 0.01328144],
       [ 0.02492655],
       [ 0.02356161],
       [ 0.01357701],
       [ 0.00916721],
       [-0.0042893 ]])

In [34]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")


Saved model to disk


In [36]:
files.download("model.json")
files.download("model.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>